<a href="https://colab.research.google.com/github/auxten/data-sdk/blob/main/demo_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/auxten/data-sdk


Cloning into 'data-sdk'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 85 (delta 25), reused 75 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 61.05 KiB | 2.77 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [2]:
%cd data-sdk
!pip install -r requirements.txt

/content/data-sdk
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0


In [4]:
from datasource import DataSource
from agent import Agent, ANTHROPIC_API_KEY

# Use a Pandas-style connection configuration approach with unified interface
# chDB can use any Python function or API as a table
revenuecat = DataSource.connect(
    "API",
    url="http://localhost:8000/v1",
    api_key="",
)

# select() method to specify fields
# Allows more granular control over returned data
users = revenuecat.collection("users").select("id, name, subscription_status")

# filter method with multiple operator support
# More expressive filtering operations with SQL-like clarity
active_users = users.filter("subscription_status", "=", "active")

print(active_users)


SELECT users.id, users.name, users.subscription_status FROM Python(reader) AS users WHERE users.subscription_status = 'active'


In [5]:

# Database connections use the same unified interface
# Consistency across different data sources reduces cognitive load
# db = DataSource.connect(
#     "postgres", host="localhost", database="mydb", user="user", password="xxx"
# )

# File source connection (CSV)
db = DataSource.connect("file", path="data/comments.csv", format="CSV")

# SQL syntax sugar while maintaining DataFrame-style interface
# Familiar SQL concepts merged with modern object chaining
comments = db.table("comments").select("*")

# Add Pandas-style relational queries
# Enhanced join operations with clear relation definition
subscribed_user_comments = active_users.join(
    comments, on={"users.id": "comments.user_id"}
)

# Pandas-style pagination and limitations
# Efficient data handling without loading unnecessary records
sample = subscribed_user_comments.limit(1000)

# Show the query plan
print("Query Plan:")
print(sample)

# Execute the query and show results
print("\nResults:")
print(sample.execute())

# Execute with different output formats
# print("\nResults as JSON:")
# print(sample.execute(output_format="JSON"))


Query Plan:
SELECT users.id, users.name, users.subscription_status, comments.* FROM Python(reader) AS users JOIN file('data/comments.csv', 'CSV') AS comments ON users.id=comments.user_id WHERE users.subscription_status = 'active' LIMIT 1000

Results:
    ┌─id─┬─name────────────┬─subscription_status─┬─user_id─┬─comment_text─────────────────────────────────────┬─comments.created_at─┬─likes─┐
 1. │  1 │ John Doe        │ active              │       1 │ Great product! Really love the features.         │ 2023-01-15 10:30:00 │     5 │
 2. │  1 │ John Doe        │ active              │       1 │ Customer support was very helpful.               │ 2023-01-17 14:20:00 │     3 │
 3. │  1 │ John Doe        │ active              │       1 │ Would recommend to others.                       │ 2023-01-21 10:00:00 │     6 │
 4. │  1 │ John Doe        │ active              │       1 │ Great value for money.                           │ 2023-01-24 14:45:00 │     7 │
 5. │  1 │ John Doe        │ active    

In [6]:
import os
if "ANTHROPIC_API_KEY" not in os.environ:
  from getpass import getpass
  anthropic_api_key = getpass("Input your anthropic api key")
  os.environ["ANTHROPIC_API_KEY"]=anthropic_api_key


# Use the question function to generate SQL from natural language
analysis = sample.question("accumulate the total comments count for each user")
print("SQL generated by Agent:")
print(analysis)


AuthenticationError: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

In [ ]:
print("Results from SQL:")
print(analysis.execute())

# Rich output format options
# Multiple export formats for different downstream needs
result = analysis.to_dataframe()  # Return pandas DataFrame
print(result)
result = analysis.to_dict()  # Return dictionary structure
print(result)


In [ ]:
analysis.plot(x="name", y="total_comments", kind="bar")